In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/content/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [3]:
X =df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [4]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X = ss.fit_transform(X)

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [6]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [7]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,batch_size=32,epochs = 50,validation_data=(X_test,y_test))

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6490 - loss: 0.6538 - val_accuracy: 0.7208 - val_loss: 0.6152
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7118 - loss: 0.6022 - val_accuracy: 0.7403 - val_loss: 0.5762
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7232 - loss: 0.5690 - val_accuracy: 0.7727 - val_loss: 0.5430
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7490 - loss: 0.5170 - val_accuracy: 0.7727 - val_loss: 0.5178
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7306 - loss: 0.5208 - val_accuracy: 0.7792 - val_loss: 0.5005
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7867 - loss: 0.4741 - val_accuracy: 0.7727 - val_loss: 0.4885
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7500 - loss: 0.4951 - val_accuracy: 0.7727 - val_loss: 0.4802
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7902 - loss: 0.4620 - val_accuracy: 0.7857 - val_loss: 0.4716
Ep

# SELECTING APPROPRIATE OPTIMIZER

In [11]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.8 MB/s eta 0:00:00


In [12]:
import kerastuner as kt

<ipython-input-12-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(16,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values =['Adam','sgd','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials =5)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'Adam'}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7686 - loss: 0.4835 - val_accuracy: 0.8052 - val_loss: 0.5070
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7621 - loss: 0.4833 - val_accuracy: 0.8117 - val_loss: 0.4984
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7430 - loss: 0.4918 - val_accuracy: 0.8117 - val_loss: 0.4938
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7569 - loss: 0.4693 - val_accuracy: 0.8117 - val_loss: 0.4905
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7720 - loss: 0.4655 - val_accuracy: 0.8117 - val_loss: 0.4865
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7669 - loss: 0.4380 - val_accuracy: 0.8052 - val_loss: 0.4835
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7895 - loss: 0.4382 - val_accuracy: 0.8117 - val_loss: 0.4820
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7815 - loss: 0.4367 - val_accuracy: 0.81

In [ ]:
def build_model(hp):
  model=Sequential()
  units = hp.Int('units',min_value = 8,max_value = 128,step=8)
  model.add(Dense(units = units,activation='relu',input_dim = 8 ))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
tuner = kt.RandomSearch(build_model,objective ='val_accuracy',max_trials=5,directory='mydir',project_name='lol')

Reloading Tuner from mydir/lol/tuner0.json


In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 59s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'units': 96}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7673 - loss: 0.4930 - val_accuracy: 0.7922 - val_loss: 0.4875
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7834 - loss: 0.4705 - val_accuracy: 0.7987 - val_loss: 0.4788
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7679 - loss: 0.4534 - val_accuracy: 0.7987 - val_loss: 0.4734
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7740 - loss: 0.4599 - val_accuracy: 0.7922 - val_loss: 0.4712
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7558 - loss: 0.4730 - val_accuracy: 0.7857 - val_loss: 0.4721
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7792 - loss: 0.4366 - val_accuracy: 0.7792 - val_loss: 0.4716
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7657 - loss: 0.4650 - val_accuracy: 0.7727 - val_loss: 0.4707
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7678 - loss: 0.4615 - val_accuracy: 0.78

# HOW TO SELECT NUMBER OF LAYERS


In [8]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(72,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu'))
  model.add(Dense(1,activation='relu'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [13]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name='lol')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 15s


In [15]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9}

In [16]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [19]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7697 - loss: 0.7783 - val_accuracy: 0.7857 - val_loss: 0.4481
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8211 - loss: 0.4073 - val_accuracy: 0.7403 - val_loss: 1.0513
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7818 - loss: 0.6019 - val_accuracy: 0.7792 - val_loss: 1.0977
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8295 - loss: 0.5812 - val_accuracy: 0.7727 - val_loss: 0.7507
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8249 - loss: 0.4275 - val_accuracy: 0.7208 - val_loss: 1.3186
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7814 - loss: 0.5213 - val_accuracy: 0.7403 - val_loss: 1.6937
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8310 - loss: 0.4507 - val_accuracy: 0.7922 - val_loss: 1.0121
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8538 - loss: 0.5112 - val_accuracy: 0.75

# TUNNING ALL IN ONE

In [31]:
def build_model(hp):
  model = Sequential()
  counter = 0

  for i in range(hp.Int('num_layers',min_value = 1,max_value=10)):
       if counter == 0:
        model.add(Dense(hp.Int('units' + str(i),min_value = 8,max_value = 128),activation=hp.Choice('activation'+ str(i),values=['relu','tanh','sigmoid'])
        ,input_dim = 8))
       else :
        model.add(
            Dense(
                hp.Int('units' + str(i),min_value = 8,max_value = 128),activation=hp.Choice('activation'+ str(i),values=['relu','tanh','sigmoid'])
            )
       )
        counter+=1
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values =['Adam','sgd','adadelta','rmsprop']),loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [32]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name='lol')

Reloading Tuner from mydir/lol/tuner0.json


In [33]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

In [34]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9}

In [37]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 10 objects could not be loaded. Example error message for object <Dense name=dense, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(8, 8), Received: value.shape=(8, 72). Target variable: <Variable path=sequential/dense/kernel, shape=(8, 8), dtype=float32, value=[[ 0.09178174 -0.08427441 -0.36545196  0.14411008  0.47805494 -0.17017215
   0.44231814 -0.05463749]
 [ 0.2824406   0.20098406 -0.15364295 -0.30708048 -0.33119285  0.16270727
  -0.20701799 -0.2752768 ]
 [-0.4612007   0.36126935  0.4513778   0.46658498 -0.00986093  0.02812302
   0.28973687 -0.11730286]
 [ 0.21332479 -0.59870225  0.3098681  -0.38470834 -0.14378744  0.25338763
  -0.35184234 -0.20566761]
 [ 0.41523772 -0.26309812  0.49258775  0.17184108 -0.41116998 -0.2431112
  -0.3581493   0.27910143]
 [ 0.18350118 -0.06807965 -0.13034177 -0.37649298 -0.08450216  0.02654809
  -0.06031138  0.06625056]
 [ 0.2631628   0.0796